In [1]:
import simpy
import random

In [ ]:
visitor_number = 200
bus_capacity = 14
bus_time = 10
bus_number = 1
farthest_station = 6
mean_time = 5
in_and_out_time = 1


def generate_visitors(env, gate: simpy.Store, visitor_number: int=200):
    for i in range(visitor_number):
        gate.put(f'visitor_{i}')
        print(f'{round(env.now,2)} - visitor_{i} arrived gate')
        t = random.expovariate(1 / mean_time)
        yield env.timeout(t)       
              
def bus(env:simpy.Environment, gate: simpy.Store, bus_id: str):
    bus_space = simpy.Resource(env, capacity = bus_capacity)    
    def process(item, target_station):
        with bus_space.request() as req:
            yield req
            env.process(bus_process(env, bus_time, item, target_station))
            print(f'{round(env.now,2)} - {item} arrived target station')
            
    while True:
        """generate the target station of each visitor"""
        temp_target = []
        for i in range(1,bus_capacity):
            temp_target.append(random.randint(1,farthest_station))
        target = []
        for i in temp_target:
            if i not in target:
                target.append(i)
        target.sort()
        """bus need back to the gate after it arrived the destination"""
        for target_station in target:
            item = yield gate.get()
            env.process(process(item, target_station))
        yield env.timeout((target[-1] * bus_time))

def bus_process(env, bus_time, people, target_station):
    while True:
        """the time of bus driving to the next station and the time of visitors getting in and out of the bus"""
        yield env.timeout((bus_time * target_station) + in_and_out_time) 

        
if __name__ == '__main__':
    env = simpy.Environment()

    gate = simpy.Store(env)
    in_elevator = simpy.Store(env)

    env.process(generate_visitors(env, gate))

    for i in range(bus_number):
        env.process(bus(env, gate, bus_id=f'bus_{i}'))

    env.run()

0 - visitor_0 arrived gate
0 - visitor_0 arrived target station
6.81 - visitor_1 arrived gate
6.81 - visitor_1 arrived target station
13.09 - visitor_2 arrived gate
13.09 - visitor_2 arrived target station
13.64 - visitor_3 arrived gate
13.64 - visitor_3 arrived target station
14.6 - visitor_4 arrived gate
14.6 - visitor_4 arrived target station
26.8 - visitor_5 arrived gate
28.58 - visitor_6 arrived gate
31.02 - visitor_7 arrived gate
31.52 - visitor_8 arrived gate
52.83 - visitor_9 arrived gate
64.92 - visitor_10 arrived gate
66.64 - visitor_11 arrived gate
74.6 - visitor_5 arrived target station
74.6 - visitor_6 arrived target station
74.6 - visitor_7 arrived target station
74.6 - visitor_8 arrived target station
74.6 - visitor_9 arrived target station
74.6 - visitor_10 arrived target station
87.68 - visitor_12 arrived gate
87.87 - visitor_13 arrived gate
97.48 - visitor_14 arrived gate
99.39 - visitor_15 arrived gate
102.22 - visitor_16 arrived gate
107.64 - visitor_17 arrived gate